In [38]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [12]:
file_path = '/content/training.1600000.processed.noemoticon.csv'

data = pd.read_csv(file_path, encoding='ISO-8859-1')

data.head()


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [14]:
col_names = ['target','id','date','flag','user','text']
data.columns = col_names
data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [16]:
data.shape

(1599999, 6)

In [17]:
data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [18]:
data['target'].value_counts()

,count
target,
4,800000
0,799999


In [19]:
data['target'] = data['target'].map({0:0,4:1})
data['target'].value_counts()

,count
target,
1,800000
0,799999


In [23]:
stremmer = PorterStemmer()


def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [stremmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [24]:
data['text']= data['text'].apply(stemming)

In [26]:
X= data['text'].values
y= data['target'].values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=2,test_size=0.2)

In [31]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [32]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9451285 stored elements and shape (1279999, 460555)>
  Coords	Values
  (0, 101428)	0.776106511775454
  (0, 435323)	0.25795527704002896
  (0, 160320)	0.3121253833533031
  (0, 405465)	0.30300944435328037
  (0, 66697)	0.3766706507364339
  (1, 241501)	0.5940640998201837
  (1, 437481)	0.8044177057380294
  (2, 12095)	0.23036388773072908
  (2, 136138)	0.24374957067347808
  (2, 137887)	0.2309633029863662
  (2, 78963)	0.6392177656235684
  (2, 47931)	0.2285282845469903
  (2, 93796)	0.13649957077984357
  (2, 444838)	0.143958429400076
  (2, 365159)	0.23006205656971884
  (2, 297569)	0.3706694800198963
  (2, 302597)	0.29708219091176463
  (2, 399829)	0.15725437757061775
  (2, 420103)	0.17516479966347792
  (3, 353954)	0.26672998005063203
  (3, 170022)	0.2539587120476497
  (3, 21200)	0.31953329131967634
  (3, 374815)	0.25983295391455086
  (3, 48723)	0.3790033032810089
  (3, 452673)	0.23991670863233922
  :	:
  (1279997, 145761)	0.150676354612

In [33]:
model = LogisticRegression()
model.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [34]:
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.77825625


In [35]:
def predict_sentiment(text):
    text = re.sub('[^a-zA-Z]',' ',text) # removing not a-z and A-Z
    text = text.lower()
    text = text.split()
    text = [stremmer.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    text = [text]
    text = vectorizer.transform(text)
    sentiment = model.predict(text)
    if sentiment == 0:
        return "Negative"
    else:
        return "Positive"

In [37]:
print(predict_sentiment("I LOVE YOU"))
print(predict_sentiment("I HATE YOU"))

Positive
Negative


In [39]:
pickle.dump(model , open('model.pkl' , 'wb'))

In [40]:
pickle.dump(vectorizer , open('vectorizer.pkl' , 'wb'))